In [2]:
import itertools

import numpy as np
import pandas as pd
from surprise import Dataset, KNNBasic
from tqdm import tqdm

In [4]:
# data = pd.read_table("ml-100k/u.data", header=None, usecols=[0, 1, 2], names=["user_id", "item_id", "rating"])
data = pd.read_table("ml-1m/ratings.dat", sep='::',header=None, usecols=[0, 1, 2], names=["user_id", "item_id", "rating"])

/usr/local/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [5]:
data

,user_id,item_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [ ]:
# ndarrayの行番号とユーザーID,列番号とアイテムIDを対応付ける辞書を作成
user_id2row_num = {}
row_num2user_id = {}
for i, user_id in enumerate(set(list(data["user_id"]))):
    user_id2row_num[user_id] = i
    row_num2user_id[i] = user_id

item_id2column_num = {}
column_num2item_id = {}
for i, item_id in enumerate(set(list(data["item_id"]))):
    item_id2column_num[item_id] = i
    column_num2item_id[i] = item_id

In [ ]:
data.head()

In [ ]:
# ユーザー×アイテムの評価行列を作成
rate_matrix = np.zeros((len(user_id2row_num), len(item_id2column_num)))
for row in tqdm(data.itertuples(), total=data.shape[0]):
    rate_matrix[user_id2row_num[row.user_id], item_id2column_num[row.item_id]] = row.rating

In [ ]:
data.query("user_id == 1").sort_values("item_id")

In [ ]:
rate_matrix

In [ ]:
# ユーザー×ユーザーの組み合わせを列挙
row_list = [i for i in range(len(row_num2user_id))]
comb = [c for c in itertools.combinations(row_list, 2)]

In [ ]:
for i in range(7):
    print(i)

In [ ]:
# それぞれのコアにできるだけ均等になるように担当の組み合わせを配布
print(len(comb))
comb_list = np.array_split(comb, 7)

In [ ]:
len(comb_list)

In [ ]:
# コサイン類似度を計算
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
sim_list = []
# for c in np.array_split(comb, 7)[0]:
for c in tqdm(comb):
    sim_dict = {}
    sim_dict["comb"] = c
    sim_dict["sim"] = cos_sim(rate_matrix[c[0]], rate_matrix[c[1]])
    sim_list.append(sim_dict)

In [ ]:
# ユーザー×ユーザー行列に反映
sim_matrix = np.zeros((len(user_id2row_num), len(user_id2row_num)))
for s in sim_list:
    sim_matrix[s["comb"][0], s["comb"][1]] = s["sim"]

In [ ]:
sim_matrix

In [ ]:
# 類似度が上位k件の行リストを作成し、それらのユーザーの評価を平均し、ソート
k = 5
predict_user_id = 1
predict_row_num = user_id2row_num[predict_user_id]


top_k_rows = np.argsort(sim_matrix[predict_row_num])[::-1][:k]

topk_mean_ratings = np.mean(rate_matrix[top_k_rows, :], axis=0)
for i in np.argsort(topk_mean_ratings)[::-1]:
    if rate_matrix[predict_row_num, i] == 0:
        print(column_num2item_id[i], topk_mean_ratings[i])
        break


# top_k_user_ids= [row_num2user_id[i] for i in top_k_rows]

In [ ]:
np.argsort(topk_mean_ratings)[::-1]

In [ ]:
kkk

In [ ]:
topk_mean_ratings

In [ ]:
top_k_user_ids

In [ ]:
# sim_matrix[0][np.argsort(sim_matrix[0])[::-1][:5]]
np.argsort(sim_matrix[0])[::-1][:5]

In [ ]:
max(sim_matrix[0])

In [ ]:
sim_matrix(rate_matrix[0], rate_matrix[4])

In [ ]:
rate_matrix[0]

In [ ]:
rate_matrix[1]

In [ ]:
data.describe()

In [ ]:
# Load the movielens-100k dataset
data = Dataset.load_builtin("ml-100k")

# Retrieve the trainset.
trainset = data.build_full_trainset()

In [ ]:
# Build an algorithm, and train it.
algo = KNNBasic()
algo.fit(trainset)

In [ ]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

In [ ]:
iter_num = 100000000
result = 0
for i in range(iter_num):
    result = result + i

In [ ]:
result

In [ ]:
ls = [i for i in range(100000)]
c = itertools.combinations(ls, 2)
combinations = [comb for comb in c]

In [ ]:
c

In [ ]:
len(combinations)